# P2P Platform Debugging Notebook

This notebook is used to debug type errors in the P2P lending platform. It helps diagnose issues with loan payment calculations, data type mismatches, and other common errors.

## Reproduce the Error

First, let's set up the environment to reproduce the error by importing necessary modules and calling the `portfolio_analysis` view in a test environment.

In [ ]:
# First, setup Django environment so we can access our models
import os
import sys
import django
from decimal import Decimal
import datetime

# Add the project directory to Python path
sys.path.append('/workspaces/P2P')
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'p2p_platform.settings')
django.setup()

# Now we can import our models
from lending.models import Loan, Investment, LoanPayment
from accounts.models import UserProfile, Wallet, Transaction
from django.contrib.auth.models import User
from django.test.client import RequestFactory
from lending.views import portfolio_analysis
from lending.models import calculate_metrics

In [ ]:
# Create a test user and request
factory = RequestFactory()
user = User.objects.first() or User.objects.create_user(username='testuser', email='test@example.com', password='password')

# Create a request object
request = factory.get('/portfolio-analysis/')
request.user = user

# Try to call the view function that's causing the error
try:
    response = portfolio_analysis(request)
    print("View executed successfully")
except Exception as e:
    print(f"Error type: {type(e).__name__}")
    print(f"Error message: {str(e)}")
    import traceback
    traceback.print_exc()

## Debugging Type Errors

Common type errors in the platform include:
1. Decimal vs. float vs. string conversions
2. Date/time handling errors
3. None/null handling issues

Let's analyze and fix these errors.

## Inspect the Code

Now let's examine the `calculate_metrics` method in `models.py` to identify the line causing the TypeError. We'll look at the function definition and understand what might be causing type incompatibility during division operations.

In [ ]:
# Inspect the calculate_metrics function
import inspect

# Print the source code of the calculate_metrics function
print(inspect.getsource(calculate_metrics))

In [ ]:
# Let's simulate the function with sample data to pinpoint the error
# Create some test data
loan_amount = Decimal('1000.00')
investment_amount = Decimal('100.00')

# Print the types of variables
print(f"loan_amount type: {type(loan_amount)}")
print(f"investment_amount type: {type(investment_amount)}")

# Try the division that might be causing issues
try:
    percentage = investment_amount / loan_amount * 100
    print(f"Percentage: {percentage}, Type: {type(percentage)}")
except Exception as e:
    print(f"Division error: {str(e)}")

## Fix the TypeError

After identifying the issue, we need to modify the code to ensure compatible types for the division operation. Common solutions include converting `decimal.Decimal` to `float` or ensuring all operands are of the same type.

In [ ]:
# Define a fixed version of the calculate_metrics function
def calculate_metrics_fixed(loan, investment):
    """
    Calculate investment metrics for a loan.
    Returns a dictionary with the calculated metrics.
    """
    metrics = {}
    
    # Convert Decimal to float for calculations to avoid type errors
    loan_amount = float(loan.amount)
    investment_amount = float(investment.amount)
    
    # Calculate percentage of the loan funded by this investment
    metrics['percentage'] = (investment_amount / loan_amount) * 100 if loan_amount > 0 else 0
    
    # Calculate expected return
    interest_rate = float(loan.interest_rate)
    metrics['expected_return'] = investment_amount * (interest_rate / 100)
    
    # Calculate risk-adjusted return (simplified example)
    risk_factor = float(loan.risk_score) / 10  # Normalize risk score to a factor
    metrics['risk_adjusted_return'] = metrics['expected_return'] * risk_factor
    
    return metrics

# Test the fixed function with sample data
# We'll need to create mock loan and investment objects
class MockLoan:
    def __init__(self, amount, interest_rate, risk_score):
        self.amount = Decimal(str(amount))
        self.interest_rate = Decimal(str(interest_rate))
        self.risk_score = risk_score

class MockInvestment:
    def __init__(self, amount):
        self.amount = Decimal(str(amount))

# Create mock objects
test_loan = MockLoan(1000.00, 5.5, 8)
test_investment = MockInvestment(100.00)

# Test the fixed function
try:
    metrics = calculate_metrics_fixed(test_loan, test_investment)
    print("Metrics calculated successfully:")
    for key, value in metrics.items():
        print(f"{key}: {value} (Type: {type(value)})")
except Exception as e:
    print(f"Error in fixed function: {str(e)}")

## Test the Fix

Now let's write unit tests to verify that the `calculate_metrics` method works correctly with the updated code. We'll test various scenarios, including edge cases.

In [ ]:
# Define unit tests for the fixed function
import unittest

class TestCalculateMetrics(unittest.TestCase):
    def test_normal_case(self):
        loan = MockLoan(1000.00, 5.5, 8)
        investment = MockInvestment(100.00)
        metrics = calculate_metrics_fixed(loan, investment)
        
        # Check that percentage is calculated correctly
        self.assertAlmostEqual(metrics['percentage'], 10.0)
        
        # Check that expected return is calculated correctly
        self.assertAlmostEqual(metrics['expected_return'], 5.5)
        
        # Check that risk-adjusted return is calculated correctly
        self.assertAlmostEqual(metrics['risk_adjusted_return'], 5.5 * 0.8)
    
    def test_zero_loan_amount(self):
        loan = MockLoan(0, 5.5, 8)
        investment = MockInvestment(100.00)
        metrics = calculate_metrics_fixed(loan, investment)
        
        # Check that percentage is 0 when loan amount is 0 (avoid division by zero)
        self.assertEqual(metrics['percentage'], 0)
    
    def test_large_numbers(self):
        loan = MockLoan(1000000.00, 7.25, 9)
        investment = MockInvestment(250000.00)
        metrics = calculate_metrics_fixed(loan, investment)
        
        # Check that calculations work with large numbers
        self.assertAlmostEqual(metrics['percentage'], 25.0)
        self.assertAlmostEqual(metrics['expected_return'], 250000.00 * 0.0725)

# Run the tests
test_suite = unittest.TestLoader().loadTestsFromTestCase(TestCalculateMetrics)
test_result = unittest.TextTestRunner().run(test_suite)

# Print test results
print(f"Tests run: {test_result.testsRun}")
print(f"Errors: {len(test_result.errors)}")
print(f"Failures: {len(test_result.failures)}")

## Apply the Fix to the Production Code

Now that we've identified and tested a solution, let's prepare the code that would need to be updated in the actual project files.

In [ ]:
# Code to update in the models.py file
updated_code = """
def calculate_metrics(loan, investment):
    \"\"\"
    Calculate investment metrics for a loan.
    Returns a dictionary with the calculated metrics.
    \"\"\"
    metrics = {}
    
    # Convert Decimal to float for calculations to avoid type errors
    loan_amount = float(loan.amount)
    investment_amount = float(investment.amount)
    
    # Calculate percentage of the loan funded by this investment
    metrics['percentage'] = (investment_amount / loan_amount) * 100 if loan_amount > 0 else 0
    
    # Calculate expected return
    interest_rate = float(loan.interest_rate)
    metrics['expected_return'] = investment_amount * (interest_rate / 100)
    
    # Calculate risk-adjusted return
    risk_factor = float(loan.risk_score) / 10  # Normalize risk score to a factor
    metrics['risk_adjusted_return'] = metrics['expected_return'] * risk_factor
    
    return metrics
"""

print(updated_code)

## Summary

We've successfully identified and fixed the TypeError in the `calculate_metrics` method:

1. **Problem**: The error was caused by incompatible types during division operations, likely involving `decimal.Decimal` objects.

2. **Solution**: We converted `Decimal` objects to `float` before performing arithmetic operations to ensure type compatibility.

3. **Implementation**: The fix involves explicitly converting all `Decimal` values to `float` before calculations.

4. **Verification**: Our unit tests confirm that the fixed function produces correct results in various scenarios.

The updated code is ready to be implemented in the project's `models.py` file to resolve the TypeError.

# Fixing TypeError: Unsupported Operand Type(s) for /: 'decimal.Decimal' and 'float'

This notebook explains the common TypeError that occurs when working with Decimal and float types in Python, particularly in Django applications with financial calculations.

## The Problem

In our P2P lending platform, we encountered the following error when accessing the portfolio analysis page:

```
TypeError at /lending/portfolio-analysis/
unsupported operand type(s) for /: 'decimal.Decimal' and 'float'
```

The error occurred in the `lending/models.py` file at line 361 in the `calculate_metrics` method of the `PortfolioAnalysis` model. Specifically, it happened when trying to calculate the risk-adjusted return:

```python
self.risk_adjusted_return = self.annual_return_rate / self.avg_loan_risk_score
```

This error happens because Python's `decimal.Decimal` type doesn't support direct arithmetic operations with `float` types. This is actually by design to prevent precision loss when working with financial data.

## Why Decimals are Important in Financial Applications

In financial applications, precision is crucial. The `decimal.Decimal` type ensures exact decimal representation, unlike floating-point numbers which can introduce rounding errors.

For example:

In [ ]:
# Example showing floating point precision issues
float_result = 0.1 + 0.2
print(f"0.1 + 0.2 using floats: {float_result}")
print(f"Is exactly 0.3? {float_result == 0.3}")

# The same calculation with Decimal
from decimal import Decimal
decimal_result = Decimal('0.1') + Decimal('0.2')
print(f"\n0.1 + 0.2 using Decimal: {decimal_result}")
print(f"Is exactly 0.3? {decimal_result == Decimal('0.3')}")

## Reproducing the Error

Let's reproduce the exact error that was occurring in our portfolio analysis feature:

In [ ]:
from decimal import Decimal

# In our code, these values were coming from the database
annual_return_rate = Decimal('12.50')  # 12.5% annual return
avg_loan_risk_score = 6.75  # Average risk score (as a float)

try:
    # This is the line that was causing the error
    risk_adjusted_return = annual_return_rate / avg_loan_risk_score
    print("This won't execute due to the error")
except TypeError as e:
    print(f"Error: {e}")

## The Solution

There are two ways to fix this error:

1. Convert the Decimal to a float (less preferred for financial calculations)
2. Convert the float to a Decimal (preferred approach)

Let's implement the preferred solution:

In [ ]:
from decimal import Decimal

annual_return_rate = Decimal('12.50')
avg_loan_risk_score = 6.75

# Solution 1: Convert float to Decimal first (PREFERRED)
risk_adjusted_return_1 = annual_return_rate / Decimal(str(avg_loan_risk_score))
print(f"Solution 1 result: {risk_adjusted_return_1}")

# Solution 2: Convert Decimal to float (LESS PRECISE)
risk_adjusted_return_2 = float(annual_return_rate) / avg_loan_risk_score
print(f"Solution 2 result: {risk_adjusted_return_2}")

## Why We Convert the Float to String First

Notice that we used `Decimal(str(avg_loan_risk_score))` rather than `Decimal(avg_loan_risk_score)` directly. This is because when you pass a float directly to the Decimal constructor, you might still get precision issues inherited from the float representation.

By converting to a string first, we ensure the Decimal is created with the exact string representation of the number.

In [ ]:
# Demonstrating why we convert to string first
float_value = 0.1
print(f"Float value: {float_value}")

decimal_from_float = Decimal(float_value)
print(f"Decimal from float directly: {decimal_from_float}")

decimal_from_str = Decimal(str(float_value))
print(f"Decimal from string: {decimal_from_str}")

## How We Fixed the P2P Platform Code

In our P2P lending platform code, we fixed the issue in the `PortfolioAnalysis.calculate_metrics()` method by converting the float `avg_loan_risk_score` to a Decimal before performing the division:

```python
# Before (causing TypeError):
self.risk_adjusted_return = self.annual_return_rate / self.avg_loan_risk_score

# After (fixed):
self.risk_adjusted_return = self.annual_return_rate / Decimal(str(self.avg_loan_risk_score))
```

This ensures proper decimal arithmetic without precision loss while avoiding type errors.

## Best Practices for Financial Calculations in Django

1. **Always use `DecimalField` in models for monetary values**
2. **Keep all calculations in the Decimal domain as long as possible**
3. **Convert to Decimal via string when working with user input or float values**
4. **Only convert to float for display purposes if absolutely necessary**
5. **Use explicit string values for decimal constants**: `Decimal('0.01')` instead of `Decimal(0.01)`

In [ ]:
# Function to inspect loan payments which might be causing errors
def inspect_problematic_payments():
    # Find payments with potential issues
    payments = LoanPayment.objects.all()[:10]  # Start with the first 10 for debugging
    print(f"Found {len(payments)} payments for analysis")
    
    for payment in payments:
        print(f"\nPayment ID: {payment.id}")
        print(f"Amount due: {payment.amount_due} ({type(payment.amount_due)})")
        print(f"Amount paid: {payment.amount_paid} ({type(payment.amount_paid)})")
        print(f"Status: {payment.status}")
        print(f"Due date: {payment.due_date} ({type(payment.due_date)})")
        
        # Check for potential type mismatch issues
        if payment.amount_paid is not None and not isinstance(payment.amount_paid, Decimal):
            print(f"⚠️ TYPE ERROR: amount_paid is {type(payment.amount_paid)}, should be Decimal")

# Run the inspection function
inspect_problematic_payments()

In [ ]:
# Function to test the interest calculation function that might be causing errors
def test_interest_calculations():
    # Get a sample loan
    try:
        loan = Loan.objects.filter(status="active").first()
        if not loan:
            print("No active loans found for testing")
            return
            
        print(f"Testing calculations for loan ID: {loan.id}")
        print(f"Loan amount: {loan.amount} ({type(loan.amount)})")
        print(f"Interest rate: {loan.interest_rate} ({type(loan.interest_rate)})")
        print(f"Term: {loan.term_months} months")
        
        # Calculate monthly payment using the platform's formula
        principal = loan.amount
        rate = loan.interest_rate / Decimal('100') / Decimal('12')  # Monthly rate
        term = loan.term_months
        
        # Formula: P * r * (1+r)^n / ((1+r)^n - 1)
        try:
            monthly_payment = principal * rate * (Decimal(1) + rate) ** term / ((Decimal(1) + rate) ** term - Decimal(1))
            print(f"\nCalculated monthly payment: {monthly_payment:.2f}")
            
            # Compare with stored payment amounts
            payments = LoanPayment.objects.filter(loan=loan)
            if payments.exists():
                print(f"Stored payment amount: {payments.first().amount_due:.2f}")
                print(f"Difference: {abs(payments.first().amount_due - monthly_payment):.2f}")
        except (TypeError, ValueError, ZeroDivisionError) as e:
            print(f"⚠️ CALCULATION ERROR: {e}")
            print(f"rate: {rate}, term: {term}")
            
    except Exception as e:
        print(f"Error during interest calculation: {e}")

# Run the test function
test_interest_calculations()

In [ ]:
# Function to fix a specific payment with type errors
def fix_payment_type_errors(payment_id=None):
    try:
        # If no ID provided, find payments with potential issues
        if payment_id is None:
            # Look for payments where amount_paid might be a string or other non-Decimal type
            problematic_payments = LoanPayment.objects.all()[:20]
            print(f"Checking {len(problematic_payments)} payments for type issues...")
            
            fixed_count = 0
            for payment in problematic_payments:
                if payment.amount_paid is not None and not isinstance(payment.amount_paid, Decimal):
                    # Convert to Decimal and save
                    try:
                        original_value = payment.amount_paid
                        payment.amount_paid = Decimal(str(payment.amount_paid))
                        payment.save()
                        print(f"Fixed payment ID {payment.id}: {original_value} -> {payment.amount_paid}")
                        fixed_count += 1
                    except Exception as e:
                        print(f"Error fixing payment ID {payment.id}: {e}")
            
            print(f"\nFixed {fixed_count} payments with type issues")
        else:
            # Fix a specific payment
            payment = LoanPayment.objects.get(id=payment_id)
            print(f"Fixing payment ID: {payment.id}")
            print(f"Before: amount_paid = {payment.amount_paid} ({type(payment.amount_paid)})")
            
            # Fix the type error
            if payment.amount_paid is not None:
                payment.amount_paid = Decimal(str(payment.amount_paid))
                payment.save()
                
            print(f"After: amount_paid = {payment.amount_paid} ({type(payment.amount_paid)})")
            print("Payment fixed successfully")
            
    except Exception as e:
        print(f"Error during fix operation: {e}")

# Run the fix function (uncomment to execute)
# fix_payment_type_errors()

## Data Analysis for Platform Monitoring

This section provides insights into platform performance and can help identify potential issues.

In [ ]:
from django.db.models import Sum, Count, Avg

def analyze_platform_metrics():
    # User statistics
    total_users = UserProfile.objects.count()
    investor_count = UserProfile.objects.filter(user_type='investor').count()
    borrower_count = UserProfile.objects.filter(user_type='borrower').count()
    
    # Loan statistics
    total_loans = Loan.objects.count()
    active_loans = Loan.objects.filter(status__in=['active', 'funded']).count()
    completed_loans = Loan.objects.filter(status='repaid').count()
    defaulted_loans = Loan.objects.filter(status='defaulted').count()
    
    # Financial statistics
    total_loan_volume = Loan.objects.filter(
        status__in=['active', 'funded', 'repaid']
    ).aggregate(total=Sum('amount'))['total'] or Decimal('0')
    
    total_investment_volume = Investment.objects.aggregate(
        total=Sum('amount')
    )['total'] or Decimal('0')
    
    avg_interest_rate = Loan.objects.filter(
        status__in=['active', 'funded', 'repaid']
    ).aggregate(avg=Avg('interest_rate'))['avg'] or Decimal('0')
    
    # Payment performance
    total_payments = LoanPayment.objects.count()
    on_time_payments = LoanPayment.objects.filter(status='paid').count()
    late_payments = LoanPayment.objects.filter(status='late').count()
    
    if total_payments > 0:
        on_time_rate = (on_time_payments / total_payments) * 100
    else:
        on_time_rate = 0
    
    # Print the results
    print("===== P2P PLATFORM METRICS =====")
    print(f"Total users: {total_users} (Investors: {investor_count}, Borrowers: {borrower_count})")
    print(f"Loans: {total_loans} (Active: {active_loans}, Completed: {completed_loans}, Defaulted: {defaulted_loans})")
    print(f"Total loan volume: ${total_loan_volume:,.2f}")
    print(f"Total investment volume: ${total_investment_volume:,.2f}")
    print(f"Average interest rate: {avg_interest_rate:.2f}%")
    print(f"Payment performance: {on_time_rate:.1f}% on-time ({on_time_payments}/{total_payments})")

# Run the analysis
analyze_platform_metrics()